#BD cases


In [1]:
import glob
import pandas as pd
pd.set_option('max_colwidth', 100)

files_ = glob.glob('../data/bdcase/*')

print(len(files_))
print(files_)



df2016 = pd.read_csv(files_[0])
(df2016[['No EVM Explanation', 'Program Code FPI', 'CIO Evaluation Comments','Bureau Name']]
['Bureau Name'].value_counts())


7
['../data/bdcase/BusinessCase-DHS-2016.csv', '../data/bdcase/Business_Case-DHS-2017.csv', '../data/bdcase/Business_Case-DHS-2019.csv', '../data/bdcase/Business_Case-DHS-2018.csv', '../data/bdcase/Business_Case-DHS-2020.csv', '../data/bdcase/Business_Case-DHS-2021.csv', '../data/bdcase/Business_Case-DHS-2022.csv']


U.S. Customs and Border Protection              18
United States Coast Guard                       14
Transportation Security Administration          10
National Protection and Programs Directorate     9
Department of Homeland Security                  8
Citizenship and Immigration Services             8
Immigration and Customs Enforcement              7
Federal Emergency Management Agency              7
Departmental Management and Operations           5
United States Secret Service                     3
Name: Bureau Name, dtype: int64

In [22]:

def craete_df(file, write=True):
    df2022 = pd.read_csv(file)
    df2022 = df2022.rename(columns={col: col.replace(' ', '') for col in list(df2022.columns)})


    id_cols = [col for col in df2022.columns if col.lower().endswith('id') or col.lower().endswith('code')] + ['UniqueInvestmentIdentifier',]

    
    df2022[id_cols] = df2022[id_cols].applymap(str)
    col_d = {}
    
    date_cols = [col for col in df2022.columns if 'date' in col.lower() ] + ['BudgetYear']
    num_cols = ['FundingPY1andPrior($M)', 'FundingPY($M)', 'FundingCY($M)',
       'FundingBY($M)', 'InvestmentStartYear', 'InvestmentEndYear','InvestmentYears', 'Evaluation(byAgencyCIO)', 'ConsecutiveMonthswithRedCIOEval',
       'NumberofchangestoBaseline']
    
    df2022['InvestmentYears'] = (df2022['InvestmentEndYear'] -  df2022['InvestmentStartYear'])

    df2022[[col for col in df2022.columns if col not in id_cols + date_cols + num_cols]]

    text_cols = ['AgencyStrategicObjectives/AgencyPriorityGoals',  'PaperworkReductionActOMBControlNumber(s)', 'ROIDescription',  'ExplanationofChangetoPYorCYFunding','CIOEvaluationComments', 'PMQualifications']
    exclude_cols = ['DataFreshness', 'fileName', 'PMName', 'AgencyName', 'DataFreshness','BureauCode', 'InvestmentType','InfrastructureManagementCategory', ]
    
    cat_cols = ['BureauName', 'InvestmentTitle', 'InvestmentSharedService', 'PlannedSharedService', 'AnalysisConclusion', ]
    col_d['category'] = cat_cols
    col_d['numeric'] = num_cols
    col_d['dates'] = date_cols
    col_d['ids'] = id_cols
    col_d['text'] = text_cols
    col_d['exclude'] = exclude_cols
    
    print(len(id_cols + date_cols + num_cols + cat_cols + text_cols))
    print(df2022.shape)
    df2022['fileName'] = file.split('/')[-1]
    print(file)
    return (
        df2022, col_d #[[col for col in df2022.columns if col not in id_cols + date_cols + num_cols + text_cols+ exclude_cols]]
    )

df1, col_d = craete_df(files_[0])


def Make_binaryflag(df, columns=[]):
    for col in columns:
        df[col+'Flag'] = df[col].map(lambda x: False if str(x).lower() == 'nan' else True)
    return df


def count_string(df, values_list=[('AgencyStrategicObjectives/AgencyPriorityGoals', ' - '),('PaperworkReductionActOMBControlNumber(s)','|')]):
    for values_ in values_list:
        df[values_[0] + 'Count'] = df[values_[0]].map(lambda x: None if pd.isnull(x) else  len(x.split(values_[-1]))).fillna(0)
    
    return df




# flags_ = (
#     Make_binaryflag(
#         df1.drop(columns=col_d['exclude'])
#                 )
# )
# count_string(
#     df1.drop(columns=col_d['exclude'] + col_d['ids'])
#             )[col_d['text'] + ['BureauName','ProgramCodeFPI']]

exr = df1.to_dict('records')[3]

print(df1.columns)

binaryflagcols = [ 'SharedService', 'PIVEnabled', 'PublicAPI',
       'OpenDataEvaluation-SystemDesign',
       'OpenDataEvaluation-EnterpriseDataInventory',
       'OpenDataEvaluation-OutputsMeetM-13-13',
       'OpenDataEvaluation-SchemaandDictionaryDocumentedandShared',
       'OpenDataEvaluation-NotApplicable-NotNeworModernizing','OpenDataEvaluation-NotYetValidatedbyCIO',]

df1[['PIVEnabled', 'PublicAPI']].applymap(lambda x: 1 if x.lower() == 'yes' else 0)

39
(89, 50)
../data/bdcase/BusinessCase-DHS-2016.csv
Index(['UniqueInvestmentIdentifier', 'BusinessCaseID', 'AgencyCode',
       'AgencyName', 'BureauCode', 'BureauName',
       'InvestmentTitle(ITPortfolio)', 'BriefSummary', 'ProgramCodeFPI',
       'EAandInnovation', 'ROIDescription', 'AccomplishmentsPY',
       'AccomplishmentsCYBY', 'SharedService', 'PIVEnabled', 'PublicAPI',
       'OpenDataEvaluation-SystemDesign',
       'OpenDataEvaluation-EnterpriseDataInventory',
       'OpenDataEvaluation-OutputsMeetM-13-13',
       'OpenDataEvaluation-SchemaandDictionaryDocumentedandShared',
       'OpenDataEvaluation-NotApplicable-NotNeworModernizing',
       'OpenDataEvaluation-NotApplicable-NotAppropriate(Comment)',
       'OpenDataEvaluation-NotYetValidatedbyCIO', 'FundingPY1andPrior($M)',
       'FundingPY($M)', 'FundingCY($M)', 'FundingBY($M)',
       'InvestmentStartYear', 'InvestmentEndYear',
       'ExplanationofChangetoPYorCYFunding', 'NoEVMExplanation',
       'Evaluation(byAgenc

,PIVEnabled,PublicAPI
0,1,0
1,0,1
2,0,0
3,1,0
4,0,0
...,...,...
84,1,1
85,1,1
86,0,0
87,0,0


In [23]:
def make_binary(df, columns=[]):
    for col, val in columns:
        df[col + 'BinaryFlag'] =(df[col].map(lambda x: -1 if pd.isnull(x) else x)
        .map(lambda x: 1 if str(x).lower()==val else 0)
                                )
    return df

exclude_cols = col_d['exclude']
#make_binary(df1, columns=[(col, 'yes') for col in binaryflagcols])
num_cols = ['FundingPY1andPrior($M)', 'FundingPY($M)', 'FundingCY($M)',
       'FundingBY($M)', 'InvestmentStartYear', 'InvestmentEndYear',
       'Evaluation(byAgencyCIO)', 'NumberofchangestoBaseline', 'BudgetYear',
       'InvestmentYears']


codes = ['AgencyCode', 'BureauCode', 'BusinessCaseID', 'fileName',]
category_cols = ['CIOEvaluationColor', 'DataFreshness']
exclude_cols

date_cols = [col for col in df1.columns if 'date' in col.lower()]

dfe = df1[[col for col in df1.columns if col not in ['UniqueInvestmentIdentifier'] 
           + col_d['ids'] #id_cols + exclude_cols+ binaryflagcols + date_cols + num_cols
          ]]
#dfe

df1[['ExplanationofChangetoPYorCYFunding', 'NoEVMExplanation',
       'Evaluation(byAgencyCIO)', 'CIOEvaluationComments',]]

,ExplanationofChangetoPYorCYFunding,NoEVMExplanation,Evaluation(byAgencyCIO),CIOEvaluationComments
0,"Changes to meet emerging security requirements, interoperability and hosting services in the DHS...",The non-developmental work is centered on Operations & Maintenance and Level of Effort support w...,4,The Department of Homeland Security (DHS) Homeland Security Information Network (HSIN) program i...
1,"COP budget was increased from 3.954 to 7.33 for COP replatforming, IT Enhancements, and hosting...",COP O&M is maintained as a small part of the General Dynamics (GD) Homeland Security Information...,2,The Department of Homeland Security (DHS) Analysis & Operations (A&O) Common Operating Picture (...
2,"The differences are based upon a significant reduction in PMO staff, a reduction in GFTEs, and a...",DROM has one contract that does not require the use of EVM. Because the Applications Operations...,3,The Immigration and Customs Enforcement (ICE) Detention and Removal Operations Modernization (DR...
3,"Despite overall DHS budget reductions, O&M costs rose due to expected cost increases for Data Ce...",There are only three IAAs and one contract currently awarded by this investment over the $20M th...,3,The Immigration and Customs Enforcement (ICE) IT Infrastructure program is in the Support life c...
4,Funding levels for PY and CY to the final FY 2015 President's Budget for those same years were r...,"N/A because the contract is O&M, however, there is cost, deliverable, and performance monitoring...",4,The Immigration and Customs Enforcement (ICE) Federal Financial Management System (FFMS) program...
...,...,...,...,...
84,N/A.,No contract(s) has (have) been awarded.,2,FEMA's National Flood Insurance (NFIP) legacy Information Technology Systems and Services (ITSS)...
85,This investment was included in the FY2013 President's Budget request but there have not been a...,EVM is not required for most of the contracts listed above because they are for firm fixed price...,3,The National Protection and Programs Directorate (NPPD) Continuous Diagnostics and Mitigation (C...
86,There are no changes from the 2015 President's Budget for PY and CY.,Not required - the program is not a development program but in steady state O&M.,3,The CBP Tethered Aerostat Radar System program is in the Support phase of its lifecycle. It is ...
87,NaN,"Earned value is not a contract requirement since TRIRIGA, being a steady state investment, is a ...",3,The CBP TRIRIGA program is in the Support phase of its lifecycle. It is considered a new progra...


In [24]:
web = spacy.load('en_core_web_lg')

NameError: name 'spacy' is not defined

In [25]:

dfes = dfe[dfe['BureauName'].map(lambda x: True if x in ['United States Coast Guard', 'Immigration and Customs Enforcement',] else False) ]
                                 
cioevalcomments = dfes['NoEVMExplanation'].map(lambda x: x.replace('\n', ' ')).tolist()
docs = list(web.pipe(cioevalcomments))
from spacy import displacy


from IPython.core.display import display, HTML
count = 0
investmentnames =dfes['InvestmentTitle(ITPortfolio)'].map(str).tolist()
for i, doc in enumerate(docs):
    print('\n\n' + investmentnames[i] + ':')
    
    html = displacy.render(doc.sents, style="ent")
    display(HTML(html))
    print('\n\n')

NameError: name 'web' is not defined

In [26]:
html = displacy.render(docs[5].sents, style="ent")
display(HTML(html))

NameError: name 'displacy' is not defined

In [503]:
t = dfes['CIOEvaluationComments'].tolist()[7]
import re
accomp = df1['AccomplishmentsPY'].tolist()[-1]
df1.to_dict('records')[-1]
explanation = df1['ExplanationofChangetoPYorCYFunding'].tolist()[-1]
print(explanation)

This is the first year of submission of an Exhibit 300 for this investment.  Therefore, there are no funding levels that have changed from the prior year.


In [505]:


dev_rex_list = ['NCT[ ]?\d{8}']


def get_match(text, rex):
    if isinstance(rex, (list, tuple, set)):
        rex = '(' + '|'.join(rex) + ')'
    result = re.findall(rex, text)
    return result


(
    df1['CIOEvaluationComments']
.map(lambda text: get_match(text=text.lower(), rex=('moderately high risk', 'high risk', 'low risk', 'medium risk',)))
.map(lambda x: '|'.join(x))

).sort_values()


df1['CIOEvaluationComments'].map(lambda x: get_match(
   str(x).lower(), rex=('moderately high risk','medium risk', 'high risk', 'low risk', 'medium risk', 'risk')))


df1['CIOEvaluationComments'].tolist()

example_text = "The Immigration and Customs Enforcement (ICE) Student and Exchange Visitor Information System (SEVIS) program is in the Support life cycle phase. Program leadership continues to capture, analyze, and develop mitigation strategies for likely risks in the current Risk Register. As of 3QFY15, the SEVIS program deployed SEVIS 6.20, Admissibility Indicator V1.4, and Performance Engineering 5. The releases position SEVIS to address action items specified in the February 2014 Acquisition Decision Memorandum (ADM) regarding system stabilization and vulnerability issues. Since the last assessment, the program has resolved 75% of the legacy system vulnerability issues."

doc = web("SEVIS is formulating an Analysis of Alternatives (AoA) to determine the solution set for closing the remaining vulnerabilities. Additionally, the AoA will also propose an alternative approach to support the SEVIS modernization initiative. The planned delivery of the AoA was scheduled for 1QFY16; however timeline has shifted with no update on delivery. The program continues to utilize Agile and Burn-Down charts for sprints resulting in high-quality cost estimates, which allows the program to maintain cost and schedule variance within OMB-acceptable range. The SEVIS Executive Steering Committee (ESC) continues to monitor program activities. An updated Operational Analysis scheduled for FY15 will address four key areas: customer results, innovation, strategic and business results and financial performance.")


doc22 = web("The Department of Homeland Security (DHS) Chief Information Officer (CIO) continues to assess the SEVIS program as a Moderately Low-Risk investment.',The National Protection and Programs Directorate (NPPD) National Security and Emergency Preparedness (NS/EP) Priority Telecom Services (PTS) program provides government agencies with communication capabilities during emergencies. There is no infrastructure for this investment, nor does it transmit data to anyone. This is a service and not a system. Both schedule and cost variance for PTS are within the acceptable OMB specified range. The July 2014 Operational Analysis findings indicated that PTS continues to provide cost-effective priority services to the NS/EP user community. Risks for budget and operations are identified and effectively managed. Performance metrics measure program operational efficiency, accessibility, key asset & Infrastructure protection and new customers and market penetration. These state that stakeholders and NS/EP customers are satisfied with PTS performance. Each measure achieved an Over Target condition.  The Department of Homeland Security (DHS) Chief Information Officer (CIO) assesses the NPPD NS/EP PTS program as a Low Risk investment.")

#! pip install allennlp==2.1.0 allennlp-models==2.1.0

doc = web(example_text)
html = displacy.render(doc.sents, style="ent")

doc1 = web("The Transportation Worker Identification Credential (TWIC) program was established pursuant to 102 of the Maritime Transportation Safety Act of 2002 (MTSA) and 104 of the Security and Accountability for Every Port Act of 2006 (SAFE Port Act) (October 2006).")

doc2= web("These statutes require DHS to prevent individuals from entering a secure area of a vessel or facility unless the individual holds a transportation security card or is accompanied by another individual who holds a card.")


doc3 = web("Contribution to Mission Delivery or Agency Management Support Functions: In accordance with the DHS Functional Capabilities and Activities Catalog, Version 1.0, dated January 2013, the TWIC program supports the PREVENTING TERRORISM AND ENHANCING SECURITY. Mission and the Screening function as listed in the Capability Portfolio.")
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro"}
displacy.render([doc1, doc2, doc3], style="dep", options=options)


#html = displacy.render([doc1, doc2, doc3], style="dep", page=True)
#display(HTML(html))


In [27]:
# Data Analyses Libraries
import unicodedata
import warnings

warnings.simplefilter('ignore')

! pip install jsonlines

# Data Analyses Libraries
import jsonlines
import pandas as pd
import requests
import re

# Importing Data plotting libraries
import matplotlib.pyplot as plt

# Other Miscallaneous Libraries
import warnings

warnings.filterwarnings('ignore')

# Import libraries

from bs4 import BeautifulSoup
import ujson

# importing plotly libraies an dqtdm
import plotly.graph_objects as go
from collections import defaultdict
from pathlib import Path

plt.rcParams['figure.figsize'] = 8, 5
plt.style.use("fivethirtyeight")
pd.options.plotting.backend = "plotly"
from tqdm.notebook import tqdm_notebook

cute_color_dict = {
    'Moonrise1': ("#F3DF6C", "#CEAB07", "#D5D5D3", "#24281A"),
    'Moonrise2': ("#798E87", "#C27D38", "#CCC591", "#29211F"),
    'Moonrise3': ("#85D4E3", "#F4B5BD", "#9C964A", "#CDC08C", "#FAD77B")
}
! pip install ujson

import numpy as np
# Importing Data plotting libraries
from requests.auth import HTTPDigestAuth

# Other Miscallaneous Libraries
import warnings

warnings.filterwarnings('ignore')
# Import libraries
import datetime
import zipfile
from collections import OrderedDict
import string
from bs4 import BeautifulSoup
import gzip




def listify(val):
    if isinstance(val, str):
        return val.split('; ')
    else:
        return val


def return1or0(val):
    if str(val) == 'nan':
        return 0
    else:
        return 1


def NullUnique(df):
    """this function displays null values"""
    dic = defaultdict(list)
    for col in df.columns:
        dic['Feature'].append(col)
        dic['NumUnique'].append(len(df[col].unique()))
        dic['NumNull'].append(df[col].isnull().sum())
        dic['%Null'].append(round(df[col].isnull().sum() / df.shape[0] * 100, 2))
    return pd.DataFrame(dict(dic)).sort_values(['%Null'], ascending=False).style.background_gradient()


def get_age(age_str):
    if 'month' in age_str.lower():
        return np.float(returnNonList(get_match(age_str, ['\d+']))) / 12

    elif 'year' in age_str.lower():
        return np.float(returnNonList(get_match(age_str, ['\d+'])))

    else:

        return returnNonList(get_match(age_str, ['\d+']))



def reformat_entites_cortellis(entities, label='SiTargets'):
    if isinstance(entities, dict):
        entities = [entities]
    try:
        result = [{'text': ent['@originalForm'], 'start': ent['@start'], 'end': ent['@end'], 'label': ent['@type']} for ent in entities
              if ent['@type'] == label]
    except Exception as e:
        print(repr(e), f'method: pipe_ents , entities : {entities} , ent_type : {label}')
        result = []
    return result


def filter_ents(entities):
    if isinstance(entities, dict):
        entities = [entities]
    seen_indices = set()
    ents_to_keep = []
    for ent in entities:
        ent_range = set(range(int(ent['@start']),
                              int(ent['@end'])+1))
        if ent_range.intersection(seen_indices):
            continue
        seen_indices = seen_indices.union(ent_range)
        ents_to_keep.append(ent)
    return ents_to_keep


def ents_to_dict(entities):
    ent_dict = {}
    for ent in entities:
        ent_type = ent.get('@type')
        if not ent_type:
            continue
        if ent_type in ent_dict.keys():
            ent_dict[ent_type].append(ent)
        else:
            ent_dict[ent_type] = [ent]
    return ent_dict

def read_jsonl(file_path):
    """Read a .jsonl file and yield its contents line by line.
    file_path (unicode / Path): The file path.
    YIELDS: The loaded JSON contents of each line.
    """
    # cnt = 0
    with Path(file_path).open('r', encoding='utf8') as f:
        for line in f:
            # cnt += 1
            try:  # hack to handle broken jsonl
                yield ujson.loads(line.strip())
            except ValueError:
                continue


dev_rex_list = ['NCT[ ]?\d{8}']


def get_match(text, rex):
    if isinstance(rex, (list, tuple, set)):
        rex = '(' + '|'.join(rex) + ')'
    result = re.findall(rex, text)
    return result


def unorder_dict(x):
    if isinstance(x, OrderedDict):
        return {k: unorder_dict(v) for k, v in x.items()}
    elif isinstance(x, list):
        return [unorder_dict(v) for v in x]
    else:
        return x


toeval = lambda x: eval(x) if isinstance(x, str) and x[0] == '[' else x
tolist = lambda x: x if isinstance(x, list) else (x if pd.isnull(x) else [x])


def get_weights_from_terms(d, average=False):
    """want to add heigher weright for words more similar to aut"""

    if not average:
        return np.sum(list(d.values()))

    else:
        n = len(t)
        return np.sum(list(d.values())) / n


def get_dict_withind_interest(d_, kw):
    return_dict = {}
    dww = d_.get('w').lower()
    if kw.lower() in dww.lower():
        return_dict[t] = kw

    t = d_.get('t')
    if strip_punctuation(kw.lower()) in strip_punctuation(t.lower()):
        return_dict[t] = kw

    return return_dict


def term_in_val(val, term):
    if term in val:
        return True
    else:
        return False


def write_jsonl(file_path, lines):
    """
    wrts a .jsonl file and dump contents.
    file_path (unicode / Path): The path to the output file.
    lines (list): The JSON-serializable contents of each line.
    """
    data = [ujson.dumps(line, escape_forward_slashes=False) for line in lines]
    Path(file_path).open('w', encoding='utf-8').write('\n'.join(data))


def make_generator(file_name, sep=","):
    """this function uses a generator"""
    qlines = (line for line in open(file_name))
    list_line = (s.rstrip().split(sep) for s in lines)
    cols = next(list_line)
    df_dicts = (dict(zip(cols, data)) for data in list_line)
    return df_dicts


def items_present_test(input_list, clist):
    """
    Check if any of the items in clist are in input_list
    :param input_list: a list to look for them in
    :param clist: things you're looking for
    :return:
    """
    return any(x in input_list for x in clist)


def lexical_diversity(my_text_data):
    """
    input is list of text data
    output gives diversity_score
    """
    word_count = len(my_text_data)
    vocab_size = len(set(my_text_data))
    diversity_score = word_count / vocab_size
    return diversity_score


def diff_bwli(li1, li2):
    """
    this fxn returns difference between two lists
    :param li1:
    :param li2:
    :return:
    """
    return (list(set(li1) - set(li2)))


def read_gz(filename, delimiter=','):
    with gzip.open(filename, 'r') as reader:
        data = pd.read_csv(reader, delimiter=delimiter)
    return data


def pipe_ents(entities, label):
    return ' | '.join(list(set([ent['text'] for ent in entities if ent['label'] == label])))


def cln(i, extent=1):
    """
    String white space 'cleaner'.
    :param i:
    :param extent: 1 --> all white space reduced to length 1; 2 --> removal of all white space.
    :return:
    """

    if isinstance(i, str) and i != "":
        if extent == 1:
            return re.sub(r"\s\s+", " ", i)
        elif extent == 2:
            return re.sub(r"\s+", "", i)
    else:
        return i


def partial_match(input_str, looking_for):
    """
    :param input_str:
    :param looking_for:
    :return:
    """

    if isinstance(input_str, str) and isinstance(looking_for, str):
        return cln(looking_for.lower(), 1) in cln(input_str.lower(), 1)
    else:
        return False


stats_dict = {
    'sum': 'TotalCount',
    'average': 'Avg',
    'mean': 'Avg',
    'median': 'Median',

}


def get_stat(df, label, aggfunc):
    name_ = stats_dict.get(aggfunc)
    return pd.pivot_table(data=df, index='authorId', values=label, aggfunc=aggfunc).rename(
        columns={label: label.split('N_')[-1].title() + name_}).round(1)


def get_counts(l):
    """
    returns number of items within a list (can be picturay items, etc)
    :param l:
    :return:
    """
    return len(l)


def normalize_dataframe(df, columns):
    """
    uses min max scaling
    :param df: pandas dataframe object
    :param columns: list of nominal features
    :return: scaled dataframe with column values processed with min max scaling
    """

    result = df.copy()
    for feature_name in columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


def get_list_if_str(val, key):
    if val == None:
        return []
    v = val.get(key)
    if isinstance(v, list):
        return v
    elif isinstance(v, str):
        return [v]
    else:
        return val


def isinList(val, list_):
    if val in list_:
        return 1
    else:
        return 0


def create_count_bucket(val):
    """creates bucket based on count"""
    if val > 0:
        return 1
    else:
        return 0


def read_delim(map_file, sep='\t'):
    """fxn to read in delim files"""
    return pd.read_csv(map_file, sep=sep)


def flatten_json(y):
    """
    this flattens_json obj
    adds begining key to beginning
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
            out[name[:-1]] = x

    flatten(y)
    return out


def cleanAge(age):
    if len(re.findall(r'\(.*\)', age)):
        return re.findall(r'\(.*\)', age)[0]
    return '(' + age + ')'


def returnNonList(val):
    if not isinstance(val, list):
        return val
    if isinstance(val, list):
        if len(val) == 1:
            return val[0]
        else:
            return '|'.join(val)


transtable = str.maketrans(dict.fromkeys(string.punctuation))


def to_string(s):
    """
    takes input s and returns a string type and encoding if necessary
    """
    try:
        return str(s)
    except:
        # Change the encoding type if needed
        return s.encode('utf-8')


def unicodeToAscii(s):
    """
    input: string
    output: string ascii
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def strip_punctuation(input_string):
    """
    cleans string by stripping punctuation
    """

    return input_string.translate(transtable)


def create_df_from_table(table):
    """takes a bs4 soup obj and returns a list of dicts"""
    l = {}
    rows = iter(table)
    for row in rows:
        values = [col.text for col in row]

        l[values[0]] = values[1]
    return l


def replace_none(val, returnval='--'):
    if val == None:
        return returnval

    else:
        return val


def read_json(fin):
    with open(fin, 'r') as f:
        dat = ujson.load(f)
    return dat


def return_soup_from_html(path_counts):
    """returns beautfiul soup obj """
    with open(path_counts, 'r') as fin:
        soup = BeautifulSoup(fin, 'html.parser')
    return soup


today = datetime.datetime.today().strftime('%Y-%m-%d')
print(f"{today}")


def chunks(l, n):
    """Yield n number of sequential chunks from l."""
    d, r = divmod(len(l), n)
    for i in range(n):
        si = (d + 1) * (i if i < r else r) + d * (0 if i < r else i - r)
        yield l[si:si + (d + 1 if i < r else d)]


def gather_categorical_col(df, n=10):
    df = df.applymap(returnNonList)
    categorical_col = []
    for column in df.columns:
        if df[column].dtype == object and len(df[column].unique()) <= n:
            categorical_col.append(column)

            print(f"{column} : {df[column].unique()}")
            print("====================================")
    return categorical_col


flatten = lambda l: [item for sublist in l for item in sublist]


def callapisimple(url_):
    """whatever url you would like to use (cortellis endpoint )"""
    resp = requests.get(url=url_, auth=HTTPDigestAuth(KEY, PASS),
                        headers={'Accept': 'application/json'})
    return resp



def returnList(val):
    if val == None:
        return []

    elif isinstance(val, list):
        return val
    else:
        return [val]

ModuleNotFoundError: No module named 'ujson'

In [507]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.rc

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo.2021-02-11.tar.gz")

def give_response(text_,predictor=predictor, question="What type of risk is the investment?"):
    response = predictor.predict(
    passage=text_,
    question=question
    )
    return response

r = give_response(t)



In [398]:
input_ = {
  "passage": "The Department of Homeland Security (DHS) Analysis & Operations (A&O) Common Operating Picture (COP) program is in the Mixed life cycle phase. Inaccurate reporting reflects the program's cost and schedule variances outside OMB acceptable thresholds.  Consequently, this will require a re-baseline of activities. The program also continues to be understaffed, which has negatively impacted program performance. Program leadership is working to reacquire manpower to adequately staff the program. Program leadership is currently working on an operational analysis (OA). The OA is expected to be approved during the next review cycle. Once the OA has been approved work will begin on updating the life cycle cost estimate (LCCE), risk management plan (RMP) and risk register.  Noting the issues significantly impacting the program the Department of Homeland Security (DHS) Chief Information Officer (CIO) downgrades the Analysis & Operations (A&O) Common Operating Picture (COP) program to a Moderately High Risk investment.",
  "question": "What type of risk is this investment?"
}

r2 = give_response(t)




In [508]:
r2.keys()

NameError: name 'r2' is not defined

In [ ]:
r2['best_span_str']

In [411]:
df1['EAandInnovation'][6]

'SEVIS is being incrementally enhanced to overcome technical / programmatic limitations in the current system, but is constrained to be developed under the O&M Phase using current infrastructure and DHS-approved functionalities per the Acquisition Decision Memorandum of 14 February, 2013. During this enhancement effort, ICE will attempt to bring SEVIS into better alignment with the ICE target EA while identifying capabilities within the enterprise that can be leveraged during the modernization. The system will also be designed to allow reuse of its core components where appropriate value has been identified.'

In [413]:
df1.to_dict('records')[6]

{'UniqueInvestmentIdentifier': '024-000005363',
 'BusinessCaseID': '143',
 'AgencyCode': '24',
 'AgencyName': 'Department of Homeland Security',
 'BureauCode': '55',
 'BureauName': 'Immigration and Customs Enforcement',
 'InvestmentTitle(ITPortfolio)': 'ICE - Student and Exchange Visitor Information System (SEVIS)',
 'BriefSummary': "The purpose of SEVIS is to fulfill a legislative mandate to electronically collect certain information from approved schools and exchange programs on nonimmigrant students, exchange visitors, and their dependents. The goal of SEVIS is to track information on nonimmigrants who are participating in the U.S. education system or designated exchange visitor program throughout the duration of their approved stay. Nonimmigrants may temporarily come to the US to study under three classes of admission: F for academic and language students, M for vocational students, and J for exchange visitors. SEVIS greatly improved efficiencies by using electronic means to captur

In [415]:
df2 = pd.read_csv(files_[2])

In [421]:
df2['']

0     HSIN is a DHS Mission Critical system that connects homeland security leaders and mission partne...
1     The purpose of the COP is to support the DHS mission of responding to threats and hazards to the...
2     SEVIS electronically collects certain information from approved schools and exchange programs on...
3     The purpose of IDENT is to provide core biometric identity services for the dissemination of ide...
4     The purpose of ADIS is to support activities to prevent illegal entry, exit, & overstay of forei...
5     The purpose of the HRIT investment is to consolidate, integrate, and modernize the Department's ...
6     The purpose of the National Flood Insurance Program (NFIP) Information Technology Systems and Se...
7     Goals, and Current or Anticipated benefits: \nThe Disaster Assistance Improvement Program (DAIP)...
8     The purpose of IPAWS is to deliver "an effective, reliable, integrated, flexible, and comprehens...
9     The LSCMS Program supports FEMA's missio

In [419]:
df2['Explanation of Change to PY or CY Funding'].dropna().tolist()

['Changes to meet emerging security requirements, interoperability and hosting services, the migration to a Cloud Solution, and the planned upgrade from SharePoint-2010 to SharePoint-2016 impact the funding levels and funding categories. Additionally O&M costs for an increasing user population effect funding levels.',
 'COP budget was increased from 3.954 to 7.33 for COP replatforming, IT Enhancements, and hosting services in DHS data center two. Major upgrades will include COOP DR, increased data feeds to NOC elements and DHS components, and RFI.',
 'N/A.',
 'N/A.',
 'No significant changes in funding levels.',
 'The Final FY15 Presidents Budget request includes two program enhancements above the FY14 enacted funding level:1) A request  for an additional $1.143M to support the implementation of the enterprise-wide Performance and Learning Management System (PALMS), and 2) A request for an additional $.0879M in support of HRIT Portfolio Management. The FY16 funding level for HRIT (appr

In [427]:
! echo '{"sentence": "PY (FY17) has been underfunded by $8.76M."}' |     allennlp predict https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz -


2022-05-13 06:34:51,619 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-05-13 06:34:51,891 - INFO - allennlp.common.file_utils - cache of https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz is up-to-date
2022-05-13 06:34:51,891 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz from cache at /Users/isabelkayumetzger/.allennlp/cache/b5f1db011cc85691a5fa2bf29e055a712261a2e5d74a74edd7da2fffc98d4ab8.4c4ac7e06ec3d85631bd26b839f90b5a375d3ceeb43e3c74f1cf4758dcee2bb3
2022-05-13 06:34:51,893 - INFO - allennlp.models.archival - extracting archive file /Users/isabelkayumetzger/.allennlp/cache/b5f1db011cc85691a5fa2bf29e055a712261a2e5d74a74edd7da2fffc98d4ab8.4c4ac7e06ec3d85631bd26b839f90b5a375d3ceeb43e3c74f1cf4758dcee2bb3 to temp dir /var/folders/58/vp01s10j0lxd0qrrcz1hnfx80000gn/T/tmp4c8akog6
2022-05-13 06:34:54,814

2022-05-13 06:35:00,115 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /var/folders/58/vp01s10j0lxd0qrrcz1hnfx80000gn/T/tmp4c8akog6
input 0:  {"sentence": "PY (FY17) has been underfunded by $8.76M."}
prediction:  {"verbs": [{"verb": "has", "description": "PY ( FY17 ) [V: has] been underfunded by $ 8.76M.", "tags": ["O", "O", "O", "O", "B-V", "O", "O", "O", "O", "O"]}, {"verb": "been", "description": "PY ( FY17 ) has [V: been] underfunded by $ 8.76M.", "tags": ["O", "O", "O", "O", "O", "B-V", "O", "O", "O", "O"]}, {"verb": "underfunded", "description": "[ARG1: PY ( FY17 )] has been [V: underfunded] [ARG2: by $ 8.76M.]", "tags": ["B-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "O", "O", "B-V", "B-ARG2", "I-ARG2", "I-ARG2"]}], "words": ["PY", "(", "FY17", ")", "has", "been", "underfunded", "by", "$", "8.76M."]}



In [423]:
pred = {"verbs": [{"verb": "Did", "description": "[V: Did] Uriah honestly think he could beat the game in under three hours ? .", "tags": ["B-V", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}, {"verb": "think", "description": "Did [ARG0: Uriah] [ARGM-ADV: honestly] [V: think] [ARG1: he could beat the game in under three hours] ? .", "tags": ["O", "B-ARG0", "B-ARGM-ADV", "B-V", "B-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "O", "O"]}, {"verb": "could", "description": "Did Uriah honestly think he [V: could] beat the game in under three hours ? .", "tags": ["O", "O", "O", "O", "O", "B-V", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}, {"verb": "beat", "description": "Did Uriah honestly think [ARG0: he] [ARGM-MOD: could] [V: beat] [ARG1: the game] [ARGM-TMP: in under three hours] ? .", "tags": ["O", "O", "O", "O", "B-ARG0", "B-ARGM-MOD", "B-V", "B-ARG1", "I-ARG1", "B-ARGM-TMP", "I-ARGM-TMP", "I-ARGM-TMP", "I-ARGM-TMP", "O", "O"]}], "words": ["Did", "Uriah", "honestly", "think", "he", "could", "beat", "the", "game", "in", "under", "three", "hours", "?", "."]}
pred.keys()

dict_keys(['verbs', 'words'])

In [431]:
input_ =  {"sentence": "PY (FY17) has been underfunded by $8.76M."}
pred = {"verbs": [{"verb": "has", "description": "PY ( FY17 ) [V: has] been underfunded by $ 8.76M.", "tags": ["O", "O", "O", "O", "B-V", "O", "O", "O", "O", "O"]}, {"verb": "been", "description": "PY ( FY17 ) has [V: been] underfunded by $ 8.76M.", "tags": ["O", "O", "O", "O", "O", "B-V", "O", "O", "O", "O"]}, {"verb": "underfunded", "description": "[ARG1: PY ( FY17 )] has been [V: underfunded] [ARG2: by $ 8.76M.]", "tags": ["B-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "O", "O", "B-V", "B-ARG2", "I-ARG2", "I-ARG2"]}], "words": ["PY", "(", "FY17", ")", "has", "been", "underfunded", "by", "$", "8.76M."]}

[v for v in pred['verbs'] if v['verb']=='underfunded']


[{'verb': 'underfunded',
  'description': '[ARG1: PY ( FY17 )] has been [V: underfunded] [ARG2: by $ 8.76M.]',
  'tags': ['B-ARG1',
   'I-ARG1',
   'I-ARG1',
   'I-ARG1',
   'O',
   'O',
   'B-V',
   'B-ARG2',
   'I-ARG2',
   'I-ARG2']}]

In [432]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/fine-grained-ner.2021-02-11.tar.gz")
def return_pred(s):
    r = predictor.predict(
    sentence=s #"Did Uriah honestly think he could beat The Legend of Zelda in under three hours?."
    )
    return r

downloading:   0%|          | 0/376524004 [00:00<?, ?B/s]

In [434]:
r = return_pred(s="The Immigration and Customs Enforcement (ICE) Student and Exchange Visitor Information System (SEVIS) program is in the Support life cycle phase. Program leadership continues to capture, analyze, and develop mitigation strategies for likely risks in the current Risk Register. As of 3QFY15, the SEVIS program deployed SEVIS 6.20, Admissibility Indicator V1.4, and Performance Engineering 5. The releases position SEVIS to address action items specified in the February 2014 Acquisition Decision Memorandum (ADM) regarding system stabilization and vulnerability issues. Since the last assessment, the program has resolved 75% of the legacy system vulnerability issues. SEVIS is formulating an Analysis of Alternatives (AoA) to determine the solution set for closing the remaining vulnerabilities. Additionally, the AoA will also propose an alternative approach to support the SEVIS modernization initiative. The planned delivery of the AoA was scheduled for 1QFY16; however timeline has shifted with no update on delivery. The program continues to utilize Agile and Burn-Down charts for sprints resulting in high-quality cost estimates, which allows the program to maintain cost and schedule variance within OMB-acceptable range. The SEVIS Executive Steering Committee (ESC) continues to monitor program activities. An updated Operational Analysis scheduled for FY15 will address four key areas: customer results, innovation, strategic and business results and financial performance.   The Department of Homeland Security (DHS) Chief Information Officer (CIO) continues to assess the SEVIS program as a Moderately Low-Risk investment.")

In [449]:
r = return_pred(df1['EAandInnovation'][3] + df1['CIOEvaluationComments'][3])

In [454]:
r = return_pred(s = "The program did not receive the full requested amount for PY (2017).  The program received $16M out of the requested $21M ($5M shortfall).")

In [455]:
r.keys()

dict_keys(['logits', 'mask', 'tags', 'words'])

In [458]:
np.unique(r['tags'])

array(['B-MONEY', 'I-MONEY', 'L-MONEY', 'O'], dtype='<U7')

In [460]:
r['words']

['The',
 'program',
 'did',
 'not',
 'receive',
 'the',
 'full',
 'requested',
 'amount',
 'for',
 'PY',
 '(',
 '2017',
 ')',
 '.',
 'The',
 'program',
 'received',
 '$',
 '16',
 'M',
 'out',
 'of',
 'the',
 'requested',
 '$',
 '21',
 'M',
 '(',
 '$',
 '5',
 'M',
 'shortfall',
 ')',
 '.']

In [509]:
df2022, col_d = craete_df(files_[-1])

37
(41, 41)
../data/bdcase/Business_Case-DHS-2022.csv


In [511]:
df2022.columns

Index(['UniqueInvestmentIdentifier', 'BusinessCaseID', 'InvestmentType',
       'InfrastructureManagementCategory', 'AgencyCode', 'AgencyName',
       'BureauCode', 'BureauName', 'InvestmentTitle',
       'AgencyStrategicObjectives/AgencyPriorityGoals', 'ROIDescription',
       'InvestmentSharedService', 'PlannedSharedService',
       'PaperworkReductionActOMBControlNumber(s)', 'PMName',
       'PMQualifications', 'FundingPY1andPrior($M)', 'FundingPY($M)',
       'FundingCY($M)', 'FundingBY($M)', 'InvestmentStartYear',
       'InvestmentEndYear', 'ExplanationofChangetoPYorCYFunding',
       'AnalysisConclusion', 'Evaluation(byAgencyCIO)',
       'CIOEvaluationComments', 'ConsecutiveMonthswithRedCIOEval',
       'DateofLastUpdatetoCIOEvaluation', 'DateofLastChangetoCIOEvaluation',
       'NumberofchangestoBaseline', 'DateofLastBaseline',
       'DateofLastChangetoActivities', 'DateofLastUpdatetoActivities',
       'DataFreshness', 'DateofLastChangetoPerformanceMetrics',
       'DateofLa

In [513]:
df2022.describe()

,FundingPY1andPrior($M),FundingPY($M),FundingCY($M),FundingBY($M),InvestmentStartYear,InvestmentEndYear,Evaluation(byAgencyCIO),ConsecutiveMonthswithRedCIOEval,NumberofchangestoBaseline,BudgetYear,InvestmentYears
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,40.000000,41.0,41.000000
mean,634.567537,53.749073,52.820171,54.457195,2007.634146,2028.243902,3.219512,1.829268,9.725000,2022.0,20.609756
std,856.392561,48.966734,51.862731,55.821558,6.628560,4.465313,0.570622,8.142182,5.002499,0.0,6.833294
min,0.000000,0.000000,2.136000,2.137000,1995.000000,2021.000000,2.000000,0.000000,1.000000,2022.0,7.000000
25%,122.403000,19.792000,17.468000,16.568000,2003.000000,2025.000000,3.000000,0.000000,5.750000,2022.0,15.000000
50%,385.098000,29.284000,28.142000,27.542000,2007.000000,2028.000000,3.000000,0.000000,10.000000,2022.0,21.000000
75%,675.838000,81.368000,76.500000,77.722000,2012.000000,2030.000000,4.000000,0.000000,13.000000,2022.0,25.000000
max,4031.198000,180.216000,206.731000,211.646000,2022.000000,2040.000000,4.000000,49.000000,21.000000,2022.0,40.000000


In [36]:
df2021, cols2021_d = craete_df(files_[-2])
df2022, cols2022_d = craete_df(files_[-1])
df2022.sample()

36
(40, 41)
../data/bdcase/Business_Case-DHS-2021.csv
36
(41, 41)
../data/bdcase/Business_Case-DHS-2022.csv


,UniqueInvestmentIdentifier,BusinessCaseID,InvestmentType,InfrastructureManagementCategory,AgencyCode,AgencyName,BureauCode,BureauName,InvestmentTitle,AgencyStrategicObjectives/AgencyPriorityGoals,...,DateofLastUpdatetoActivities,DataFreshness,DateofLastChangetoPerformanceMetrics,DateofLastUpdatetoPerformanceMetricActuals,BudgetYear,DateInvestmentFirstSubmitted,DateofLastInvestmentDetailUpdate,InvestmentAutoSubmissionDate,InvestmentYears,fileName
10,024-000005053,314,01 - Major IT,01 - Not Applicable,24,Department of Homeland Security,58,U.S. Customs and Border Protection,CBP - Automated Commercial Environment (ACE),"024SO18227 - Not Available, 024SO20317 - Enforce U.S. Trade Laws and Facilitate Lawful Internati...",...,7/30/2021 18:03,Update Required,1/19/2021 16:49,7/30/2021 18:03,2022,6/30/2009 0:00,7/30/2021 18:03,6/30/2009 0:00,25,Business_Case-DHS-2022.csv


In [34]:
df2021[cols2021_d['numeric'] + cols2021_d['text']]
yraw = df2021['AnalysisConclusion']

In [39]:
combined_2122 = pd.concat([df2021, df2022])

In [41]:
combined_2122[num_cols]

,FundingPY1andPrior($M),FundingPY($M),FundingCY($M),FundingBY($M),InvestmentStartYear,InvestmentEndYear,Evaluation(byAgencyCIO),NumberofchangestoBaseline,BudgetYear,InvestmentYears
0,1589.279,88.943,81.368,81.465,1997,2029,3,6.0,2021,32
1,370.941,30.719,29.193,29.805,2006,2028,4,8.0,2021,22
2,1220.763,110.337,110.836,112.932,2001,2025,3,5.0,2021,24
3,1781.726,166.155,128.588,92.120,2006,2034,3,13.0,2021,28
4,89.112,23.152,19.792,19.792,2004,2028,4,8.0,2021,24
...,...,...,...,...,...,...,...,...,...,...
36,96.379,28.773,3.996,4.006,2015,2030,3,4.0,2022,15
37,234.170,61.000,25.500,25.500,2017,2031,2,3.0,2022,14
38,101.170,106.798,49.249,42.022,2019,2030,3,1.0,2022,11
39,0.000,14.840,80.054,27.493,2018,2032,3,1.0,2022,14


In [45]:
y_ = 'Evaluation(byAgencyCIO)'


combined_2122.describe().columns

Index(['FundingPY1andPrior($M)', 'FundingPY($M)', 'FundingCY($M)',
       'FundingBY($M)', 'InvestmentStartYear', 'InvestmentEndYear',
       'Evaluation(byAgencyCIO)', 'ConsecutiveMonthswithRedCIOEval',
       'NumberofchangestoBaseline', 'BudgetYear', 'InvestmentYears'],
      dtype='object')

In [47]:
X = combined_2122[['FundingPY1andPrior($M)', 'FundingPY($M)', 'FundingCY($M)',
       'FundingBY($M)', 'InvestmentStartYear', 'InvestmentEndYear',
      'ConsecutiveMonthswithRedCIOEval',
       'NumberofchangestoBaseline', 'BudgetYear', 'InvestmentYears']]

y  =combined_2122[[ 'Evaluation(byAgencyCIO)']]


In [ ]:
X

Reusing dataset imdb (/Users/isabelkayumetzger/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [51]:
explainer = shap.Explainer(classifier)
shap_values = explainer(short_data[:2])
shap.plots.text(shap_values[:,:,"POSITIVE"])

NameError: name 'shap' is not defined